<a href="https://colab.research.google.com/github/mike1336git/colab_notebook/blob/main/with_js/js034_harmonicsDMC3D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### simulator( html + css + js ) + control( python )

In [1]:
#@title js034_harmonicsDMC3D / def exec_html_js() ... exec me first

# def exec_html_js()

import IPython
from IPython.display import display, HTML
from google.colab.output import eval_js

def exec_html_js():
  htm = HTML('''


<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<title>js034_harmonicsDMC3D</title>
<script type="text/javascript">

// %%%%%%%%%%%%%%%%%%%%  javaScript  %%%%%%%%%%%%%%%%%%%%

'use strict';

/* --------------------
//
//  js034_harmonicsDMC3D
//    Copyright(C) 2017-2023 Mitsuru Ikeuchi
//    Released under the MIT license ( https://opensource.org/licenses/MIT )
//
//    ver 0.0.0  2017.06.07 created, last updated on 2018.11.18
//    ver 0.0.1  2019.01.16 v1, last updated on 2021.05.25
//    ver 0.0.2  2021.10.31 v2, last updated on 2021.10.31
//    ver 0.0.3  2023.03.21 v3, last updated on 2023.08.21
//
// -------------------- diffusion Monte-Carlo method 3D
//
// DMC algorithm : I. Kosztin et. al.; e-print arXiv:physics/9702023v1 (1997)
//                 "Introduction to the Diffusion Monte Carlo Method"
// (1) replica walk
//     x -> x + sqrt(dt)*GaussianRandom
// (2) branch: replica birth-death (branching) processes
//     weight function W
//       W = exp( -(V-ER)*dt/hbar ) --> Wi nearly= 1-(Vi-Ei)*dt/h
//       m = int( wi + random[0-1] )
//     if (m=0)  death
//     if (m=1)  alive
//     if (m=2)  alive + birth one replica
//     if (m>=3) alive + birth two replica
// (3) count: number of replicas N1
// (4) energy calc
//     ER(k+1) = ER(k) + (hbar/dt)(1-N1/N0)
//        (N0: target number of replica)
// goto (1)
//
// --------------------
*/

const harmonicsDMC3D = (function(){ // ====================  harmonicsDMC3D Module  ====================

	// au: atomic unit hBar=1,e=1,me=1,a0=1
	const g_auLength = 5.29177211e-11;			// (m) 1(au) = auLength (m)
	const g_auTime = 2.418884326e-17;			// (s) 1(au) = auTime (s)
	const g_auEnergy = 4.35974465e-18;			// (J) 1(au) = auEnergy (J)
	const g_au2eV = 27.211386;					// (eV) 1(au) = 27.211386 (eV)
	const g_hbar = 1.0;							// hbar = 1 in au(atomic unit)

	let g_count = 0;
	let g_NNr = 2000;							// number of replica
	let g_NNb = 40;								// max number of box division 0:x=-5.0 50:x=0.0 100:x=+5.0
	let g_sysTime = 0.0;						// (au) system time
	let g_dt = 0.1;								// (au) time division
	let g_dx = 0.5;								// (au) x-division, boxSize = NNb*dx
	let g_dy = 0.5;								// (au) y-division, boxSize = NNb*dy
	let g_dz = 0.5;								// (au) z-division, boxSize = NNb*dz
	let g_referenceEnergy = 0.0;				// (au) reference energy of the system
	let g_energy = g_referenceEnergy;			// (au) system energy
	let g_energy100 = g_energy;
	let g_energy10000 = g_energy;
	let g_NN0 = g_NNr/2;						// memory of NN1
	let g_NN1 = g_NN0;							// number of alive replicas
	let g_NN00 = 1000;							// initial number of alive replicas
	let g_vIndex = 0;							// potential index 0:parabolic 1:well 2:well 2

	const g_repStatus = dimInt1( g_NNr );		// status of i-th replica
	const g_replica = dim2( g_NNr, 3 );			// replica[i][0] x-coordinate of i-th replica
	const g_box = dimInt3( g_NNb, g_NNb, g_NNb ); // box[ix][iy][iz] number of alive replica
	const g_psi = dim3( g_NNb, g_NNb, g_NNb );	// wave function psi[ix][iy][iy]

	function dimInt1( n ) {
		return new Int32Array( n );
	}

	function dim2( ni, nj ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = new Float64Array( nj );
		}
		return a;
	}

	function dim3( ni, nj, nk ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = [];
			for (let j=0; j<nj; j++) {
				a[i][j] =  new Float64Array( nk );
			}
		}
		return a;
	}

	function dimInt3( ni, nj, nk ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = [];
			for (let j=0; j<nj; j++) {
				a[i][j] =  new Int32Array( nk );
			}
		}
		return a;
	}


	// --------------------  set initial condition  --------------------

	function setInitialCondition( theme ) {
		const nnr=g_NNr;
		g_vIndex = theme;
		g_count = 0;
		g_sysTime = 0.0
		for (let i=0; i<nnr; i++) {
			g_repStatus[i] = 0;
		}
		const n0 = nnr/2;
		for (let i=0; i<n0; i++) {
			g_repStatus[i] = 1;
			g_replica[i][0] = 4.0*Math.random()-2.0;
			g_replica[i][1] = 4.0*Math.random()-2.0;
			g_replica[i][2] = 4.0*Math.random()-2.0;
		}
		g_NN0 = g_NN00;
		g_NN1 = g_NN0;
		g_referenceEnergy = 0.0;
	}

	function averagedPotential() {
		const nnr=g_NNr;
		let s = 0.0, n = 0;
		for (let i=0; i<nnr; i++) {
			if (g_repStatus[i]==1) {
				n += 1;
				s += potential(g_replica[i][0],g_replica[i][1],g_replica[i][2]);
			}
		}
		return s/n;
	}


	// --------------------  evolve DMC  --------------------

	function evolveDMC( nCalc ) {
		for (let i=0; i<nCalc; i++) {
			walkReplica();
			g_count++;
		}
	}

	function walkReplica() {
		const nnr=g_NNr;
		g_sysTime = g_sysTime + g_dt
		const sqrtdt = Math.sqrt(g_dt);

		// (1) replica walk
		for (let i=0; i<nnr; i++) {
			if (g_repStatus[i]==1) {
				g_replica[i][0] += sqrtdt*GaussianRandom();
				g_replica[i][1] += sqrtdt*GaussianRandom();
				g_replica[i][2] += sqrtdt*GaussianRandom();
			}
		}

		// (2) branch
		for (let i=0; i<nnr; i++) {
			if (g_repStatus[i]==1) {
				birthDeathProcess(i);
			}
		}
		for (let i=0; i<nnr; i++) {
			if (g_repStatus[i]==2) {
				g_repStatus[i] = 1;
			}
		}

		// (3) count
		g_NN1 = numberOfReplica();

		// (4) energy
		const er = g_referenceEnergy + g_hbar/g_dt*(1.0-(g_NN1+1.0)/(g_NN0+1.0)*(g_NN1+10.0*nnr)/(g_NN00+10.0*nnr));
		g_NN0 = g_NN1;
		g_referenceEnergy = er;
		g_energy100 = 0.99*g_energy100 + 0.01*g_referenceEnergy;
		if (g_count>500) {
			g_energy = 0.999*g_energy+0.001*g_referenceEnergy;
		} else {
			g_energy = g_energy100;
		}
		if (g_count>5000) {
			g_energy10000 = 0.9999*g_energy10000+0.0001*g_referenceEnergy;
		} else {
			g_energy10000 = g_energy;
		}

		setPsi(1);
	}

	function GaussianRandom() {
		//Box-Muller transform
		return Math.sqrt(-2*Math.log(1-Math.random()))*Math.sin(2*Math.PI*Math.random());
	}

	function birthDeathProcess(i) {
		const xi = g_replica[i][0], yi = g_replica[i][1], zi = g_replica[i][2];
		const mn = Math.floor(weightFunction(xi,yi,zi)+Math.random());
		if (mn==0) {
			g_repStatus[i] = 0; // death replica
		} else if (mn==2) {
			birthReplica(i);
		} else if (mn>=3) {
			birthReplica(i);
			birthReplica(i);
		}
	}

	function weightFunction(xn,yn,zn) {
		return Math.exp(-(potential(xn,yn,zn,g_vIndex)-g_referenceEnergy)*g_dt/g_hbar);
	}

	function potential(x,y,z,vIndex) {
		let ret = 0;
		if (vIndex==0) { // parabolic
			ret = 0.5*(x*x+y*y+z*z);
		} else if (vIndex==1) { // well
			ret = (Math.sqrt(x*x+y*y+z*z)<3.0) ? 0.0 : 20.0;
		} else if (vIndex==2) { // H like
			const r = Math.max(Math.sqrt(x*x+y*y+z*z),0.025);
			ret = -1.0/r;
		}  else if (vIndex==3) { // ring
			const rxy = Math.sqrt(x*x+y*y)
			ret = (2.0<rxy && rxy<4.0 && Math.abs(z)<1.0 ) ? 0 : 20.0 ;
		}
		return ret;
	}

	function birthReplica(i) {
		const ip = seekSpace(i);
		if (ip>=0) {
			g_repStatus[ip] = 2; // marked
			g_replica[ip][0] = g_replica[i][0];
			g_replica[ip][1] = g_replica[i][1];
			g_replica[ip][2] = g_replica[i][2];
		}
	}

	function seekSpace(istart) {
		const nnr=g_NNr;
		let ip = -1;
		for (let i=1; i<nnr; i++) {
			if (g_repStatus[(i+istart)%nnr]==0) {
				ip = (i+istart)%nnr;
				break;
			}
		}
		return ip;
	}

	function numberOfReplica() {
		const nnr=g_NNr;
		let n = 0;
		for (let i=0; i<nnr; i++) {
			if (g_repStatus[i]==1) n+=1;
		}
		return n;
	}

	function setPsi(sw) {
		const nnr=g_NNr,nnb=g_NNb;
		for (let ix=0; ix<nnb; ix++) {
			for (let iy=0; iy<nnb; iy++) {
				for (let iz=0; iz<nnb; iz++) {
					 g_box[ix][iy][iz] = 0;
				}
			}
		}
		for (let i=0; i<nnr; i++) {
			if (g_repStatus[i]==1) {
				const ix = Math.floor(nnb/2+g_replica[i][0]/g_dx+0.5);
				const iy = Math.floor(nnb/2+g_replica[i][1]/g_dy+0.5);
				const iz = Math.floor(nnb/2+g_replica[i][2]/g_dz+0.5);
				if (ix>=0 && ix<nnb && iy>=0 && iy<nnb && iz>=0 && iz<nnb) g_box[ix][iy][iz] += 1;
			}
		}
		if (sw==1) {
			for (let ix=0; ix<nnb; ix++) {
				for (let iy=0; iy<nnb; iy++) {
					for (let iz=0; iz<nnb; iz++) {
						g_psi[ix][iy][iz] = 0.99*g_psi[ix][iy][iz]+0.01*g_box[ix][iy][iz];
					}
				}
			}
		} else {
			for (let ix=0; ix<nnb; ix++) {
				for (let iy=0; iy<nnb; iy++) {
					for (let iz=0; iz<nnb; iz++) {
						g_psi[ix][iy][iz] = g_box[ix][iy][iz];
					}
				}
			}
		}
	}


	// --------------------  public  --------------------

	return {
		init:			setInitialCondition,	// setInitialCondition( vIndex )
		evolve:			evolveDMC,				// evolveDMC( nCalc )

		getAUinSI:		function() { return [ g_auLength, g_auTime, g_auEnergy, g_au2eV ]; },
		getSysParam:	function() { return [ g_NNr, g_NNb, g_dx, g_dt ]; },
		getNow:			function() { return [ g_count, g_energy100, g_energy, g_energy10000, g_NN1 ]; },
		getRepStatus:	function(ir) { return g_repStatus[ir]; },
		getRepPos:		function(ir) { return [ g_replica[ir][0], g_replica[ir][1], g_replica[ir][2] ]; },
		getPsi:			function(ix,iy,iz) { return g_psi[ix][iy][iz]; },
		getVext:		function(ix,iy,iz) {
							let x = (ix-g_NNb/2)*g_dx, y = (iy-g_NNb/2)*g_dx, z = (iz-g_NNb/2)*g_dx;
							return potential( x, y, z, g_vIndex );
						},
	};

})(); // ====================  hermonicsDMC3D end  ====================


const js034 = (function(){ // ====================  js Module  ====================

	const theModule = harmonicsDMC3D;
	const xCanvasSize = 480;	// in pixel
	const yCanvasSize = 480;	// in pixel
	let canvas;					// canvas2d
	let ctx;

	let v_vIndex = 0; // 0:parabolic, 1:well, 2:H like, 3:Na like empty core
	let v_nCalc = 1;

	let p_NNr, p_NNb, p_dx, p_dt; // = theModule.getSysParam();
	let sysTime, energy100, energy1000, energy10000, aliveReplicas;

	let dispMode = 1;
	let resetFlag = true;
	let pauseFlag = false;
	let stepFlag = false;
	let inStepFlag = false;
  let breakFlag = false;


	function main() {
		resetFlag = true;
		setCanvas( 'canvas_box', xCanvasSize, yCanvasSize );
		initDom();
		viewHome();

		animate();

		function setCanvas( canvasID, width, height ) {
			canvas = document.getElementById( canvasID );
			canvas.width  = width;
			canvas.height = height;
			ctx = canvas.getContext('2d');
			ctx.font = "16px 'sans-serif'";
			ctx.textBaseline = "bottom";
			ctx.textAlign = "left";
			ctx.lineWidth = 1;
			g3d.setMouseOnCanvas( canvas ); // 3D graphics
		}
	}


	function animate() {
    if ( breakFlag ) return;

		if ( resetFlag ) {
			resetFlag = false;
			theModule.init( v_vIndex );
			[ p_NNr, p_NNb, p_dx, p_dt ] = theModule.getSysParam();
			// g3d.init( NNx, NNy, NNz, dx, xCanvasSize, yCanvasSize, xBoxSize, yShift );
			g3d.init( p_NNb, p_NNb, p_NNb, p_dx, xCanvasSize, yCanvasSize, 300, 20 );
		}

		if ( !pauseFlag ) {
			theModule.evolve( v_nCalc );
		} else if ( pauseFlag && stepFlag ) {
			stepFlag = false;
			theModule.evolve( v_nCalc );
			inStepFlag = true;
		}

		draw( ctx, dispMode );

		requestAnimationFrame(animate);
	}


	// --------------------  draw  --------------------

	function draw( ctx, dispMode ) {
		const NNx = p_NNb, NNy = p_NNb, NNz = p_NNb, dx = p_dx;
		const xp = 80, yp = 60, xtabp = 320, jc = NNy/2, kc = NNz/2;

		let auLength, auTime, auEnergy, au2eV;
		[ auLength, auTime, auEnergy, au2eV ] = theModule.getAUinSI();
		[ sysTime, energy100, energy1000, energy10000, aliveReplicas ] = theModule.getNow();

		// clear
		ctx.clearRect(0, 0, xCanvasSize, yCanvasSize);

		if (dispMode==0) {
			dispText( "2D psi(x,y,0)" );
			drawPsi2D( ctx, xp, yp );

		} else if (dispMode==1) {
			dispText( "3D alive replicas" );
			drawReplicas3D(ctx, 0.0 );

		} else if (dispMode==2) {
			dispText( "3D psi(x,y,z)" );
			const zFunc = function(i,j,k) { return 3.0*theModule.getPsi(i,j,k); };
			// g3d.drawField3D( ctx, rotAngle, fieldFunc, colorMode )
			g3d.drawField3D( ctx, 0.0, zFunc, 1 );
		} else if (dispMode==3) {
			dispText( "3D density(x,y,z)" );
			const zFunc = function(i,j,k) { const psi=theModule.getPsi(i,j,k); return 1.0*psi*psi; };
			// g3d.drawField3D( ctx, rotAngle, fieldFunc, colorMode )
			g3d.drawField3D( ctx, 0.0, zFunc, 1 );
		} else if (dispMode==4) {
			dispText( "grid2d psi(x,y,0)" );
			const zFunc = function(i,j) { return 1.0*theModule.getPsi(i,j,kc); };
			// g3d.drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
			g3d.drawGrid2D( ctx, 0.0, zFunc, 0.5, 1 );
		} else if (dispMode==5) {
			dispText( "density(x,y,0)" );
			const zFunc = function(i,j) { const psi=theModule.getPsi(i,j,kc); return 0.2*psi*psi; };
			// g3d.drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
			g3d.drawGrid2D( ctx, 0.0, zFunc, 0.5, 1 );
		} else if (dispMode==6) { // 3D view - Vext(x,y)
			dispText( "grid2d external potential: Vext(x,y,0)" );
			const zFunc = function(i,j) { return 0.1*theModule.getVext(i,j,kc); };
			// g3d.drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
			g3d.drawGrid2D( ctx, 0.0, zFunc, "#00bb00", 1 );
		}

		ctx.fillStyle = "#888888";
		ctx.fillText(`box = ${NNx*dx}x${NNy*dx}x${NNz*dx} (au)`, 20, yCanvasSize-30);
		ctx.fillText(`alive replicas = ${aliveReplicas }`, 260, yCanvasSize-30);
		ctx.fillText(`time = ${sysTime.toFixed(1)} (au)`, 20, yCanvasSize-10);
		ctx.fillText(`Energy = ${energy1000.toFixed(6)} (au)`, 260, yCanvasSize-10);

		function dispText( str ) {
			ctx.fillStyle = "#888888";
			ctx.fillText( str, 20, yCanvasSize-50 );
		}
	}

	function drawPsi2D( ctx, xp, yp ) {
		const nnx=p_NNb, nny=p_NNb, sc=8, kc=p_NNb/2, vmag=5.0, pmag = 200.0;

		ctx.strokeStyle = "#888800";
		ctx.strokeRect( xp, yp, 320, 320 );

		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				const g = Math.min( Math.floor(theModule.getVext(i,j,kc)*vmag), 255 );
				const p = Math.min( Math.floor(theModule.getPsi(i,j,kc)*pmag), 255 );
				ctx.fillStyle = "rgb("+p+","+g+","+p+")";
				ctx.fillRect(i*sc+xp,(nny-j-1)*sc+yp,sc,sc);
			}
		}
	}

	function drawReplicas3D( ctx, rotAngle ) {
		let sc, xp, yp; [ sc, xp, yp ] = g3d.scxpypFunc();
		const nnr=p_NNr, nnx=g_NNx, nny=g_NNy, nnz=g_NNz, dx=g_dx, r=1.0, color="#00ffff";
		const cx0=0.5*nnx*dx, cy0=0.5*nny*dx, cz0=0.5*nnz*dx;

		g3d.set3DRotateXY(rotAngle);
		g3d.plotFarEdge(ctx, sc,xp,yp,"#444400"); // dark yellow

		for (let i=0; i<nnr; i++) {
			if ( theModule.getRepStatus(i)==0 ) continue; // 0:dead, 1:alive
			let x, y, z; [ x, y, z ] = theModule.getRepPos(i);
			const xi = x + cx0, yi = -y + cy0, zi = z + cz0;
			g3d.drawRotatedDisc(ctx, xi,yi,zi,r,color,sc,xp,yp); //(x,y,z,r,color,mag,xp,yp)
		}

		g3d.plotNearEdge( ctx, sc, xp, yp, "#999900"); //yellow

		ctx.font = "12px 'sans-serif'";
		ctx.fillStyle = "#888888";
		ctx.fillText(`Ax=${(g3d.Ax*180/Math.PI).toFixed(1)}, Ay=${(g3d.Ay*180/Math.PI).toFixed(1)}`, 10, 15);
		ctx.font = "16px 'sans-serif'";
	};


	// --------------------  graphics 3D (field) module  --------------------
	//
	// ver 0.0.1  2018.12.16  last updated on 2023.03.01
	// ver 0.0.2  2023.03.03  last updated on 2023.06.01

	let g_NNx, g_NNy, g_NNz, g_dx, g_dy, g_dz, g_xCanvasSize, g_yCanvasSize, g_xBoxSize, g_yShift;

	const g3d = {};				// namespace of graphic 3D module

	g3d.mouseDownFlag = 0;		// 1:on mouse down, 0:else
	g3d.x_mouse = 0;			// x-position of mouse
	g3d.y_mouse = 0;			// y-position of mouse
	g3d.x0_mouse = 0;			// drag-started x-position of mouse
	g3d.y0_mouse = 0;			// drag-started y-position of mouse
	g3d.zoom = 1.0;

	g3d.xMax = 0.0;				// x-length of box
	g3d.yMax = 0.0;				// y-length of box
	g3d.zMax = 0.0;				// z-length of box
	g3d.cx0 = 0.0;				// x-component of rotate center
	g3d.cy0 = 0.0;				// y-component of rotate center
	g3d.cz0 = 0.0;				// z-component of rotate center
	g3d.Ax = -Math.PI/15.0;		// rotate angle around x-axis
	g3d.Ay = -Math.PI/15.0;		// rotate angle around y-axis
	g3d.ddAy = 0.0;				// Ay change rate for auto-rotate: eg. dday=0.5*Math.PI/180
	g3d.cosAx = 0.0;			// cosAx=Math.cos(Ax)
	g3d.sinAx = 0.0;			// sinAx=Math.sin(Ax)
	g3d.cosAy = 0.0;			// cosAy=Math.cos(Ay)
	g3d.sinAy = 0.0;			// sinAy=Math.sin(Ay)

	g3d.xApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.yApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.zApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.pxApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.pyApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.pzApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.boxApex = [[0,0,0], [1,0,0], [0,1,0], [1,1,0], [0,0,1], [1,0,1], [0,1,1], [1,1,1] ];
	g3d.boxEdge = [[0,1,9], [0,2,9], [0,4,9], [1,3,9], [1,5,9], [2,3,9],
					[2,6,9], [3,7,9], [4,5,9], [4,6,9], [5,7,9], [6,7,9] ];

	//--- set mouse on canvas

	// g3d.setMouseOnCanvas( canvas );
	g3d.setMouseOnCanvas = function( canvas ) {
		canvas.addEventListener('mousemove', g3d.mouse_move);
		canvas.addEventListener('mousedown', g3d.mouse_down);
		canvas.addEventListener('mouseup', g3d.mouse_up);
		//canvas.addEventListener("mousewheel", g3d.mouseWheel);
	};

	g3d.mouse_move = function(e) {
		const pi = Math.PI;

		if (g3d.mouseDownFlag==1) {
			g3d.x_mouse = e.clientX;
			g3d.y_mouse = e.clientY;
			g3d.Ay = g3d.Ay + 0.5*(g3d.x_mouse-g3d.x0_mouse)*pi/180;
			if (g3d.Ay<-pi) g3d.Ay += 2*pi;
			if (g3d.Ay>pi) g3d.Ay += -2*pi;
			//g3d.Ax = g3d.Ax + 0.5*(g3d.y_mouse-g3d.y0_mouse)*pi/180;
			g3d.Ax = g3d.Ax - 0.5*(g3d.y_mouse-g3d.y0_mouse)*pi/180;
			if (g3d.Ax<-0.5*pi) g3d.Ax = -0.5*pi;
			if (g3d.Ax>0.5*pi) g3d.Ax = 0.5*pi;
			g3d.x0_mouse = g3d.x_mouse;
			g3d.y0_mouse = g3d.y_mouse;
		}
	};

	g3d.mouse_down = function(e) {
		if (g3d.mouseDownFlag==0) {
			g3d.x0_mouse = e.clientX;
			g3d.y0_mouse = e.clientY;
			g3d.x_mouse = g3d.x0_mouse;
			g3d.y_mouse = g3d.y0_mouse;
			g3d.mouseDownFlag = 1;
		}
	};

	g3d.mouse_up = function(e) {
		if (g3d.mouseDownFlag==1) {
			g3d.mouseDownFlag = 0;
		}
	};

	g3d.mouseWheel = function(e) {
		g3d.deltaY = e.deltaY;
		if ( g3d.deltaY > 0 ) g3d.zoom *= 0.95;
		else if ( g3d.deltaY < 0 ) g3d.zoom *= 1.05;
		if ( g3d.zoom<0.5 ) g3d.zoom = 0.5;
		if ( g3d.zoom>2.0 ) g3d.zoom = 2.0;
	};

	//--- 3D graphics aid

	// g3d.init( NNx, NNy, NNz, dx, xCanvasSize, yCanvasSize, xBoxSize, yShift );
	g3d.init = function( NNx, NNy, NNz, dx, xCanvasSize, yCanvasSize, xBoxSize, yShift ) {
		g_NNx = NNx; g_NNy = NNy; g_NNz = NNz;
		g_dx = dx; g_dy = dx; g_dz = dx;
		g_xCanvasSize = xCanvasSize; g_yCanvasSize = yCanvasSize;
		g_xBoxSize = (xBoxSize==undefined) ? 300 : xBoxSize;
		g_yShift = (yShift==undefined) ? 20 : yShift;
		g3d.setSize();
	}

	g3d.setSize = function() {
		g3d.xMax = g_NNx*g_dx;		// x-length of box
		g3d.yMax = g_NNy*g_dy;		// y-length of box
		g3d.zMax = g_NNz*g_dz;		// z-length of box
		g3d.cx0 = 0.5*g3d.xMax;		// x-component of rotate center
		g3d.cy0 = 0.5*g3d.yMax;		// y-component of rotate center
		g3d.cz0 = 0.5*g3d.zMax;		// z-component of rotate center
	};

	// g3d.setRotateAngle( AxInDegree, AyInDegree );
	g3d.setRotateAngle = function( AxInDegree, AyInDegree ) {
		g3d.Ax = AxInDegree*Math.PI/180.0;
		g3d.Ay = AyInDegree*Math.PI/180.0;
	};

	// g3d.scxpypFunc();
	g3d.scxpypFunc = function() {
		const xBoxSize = g_xBoxSize;
		const xCenter = g_xCanvasSize/2, yCenter = g_yCanvasSize/2-g_yShift, yBoxSize = (xBoxSize/g_NNx)*g_NNy;
		const xp = xCenter - (xBoxSize/2)*g3d.zoom, yp = yCenter - (yBoxSize/2)*g3d.zoom; // g3d param
		const sc = xBoxSize/(g_NNx*g_dx)*g3d.zoom;
		return [ sc, xp, yp ];
	}

	g3d.set3DRotateXY = function(rotateRateOfAyInDegree) {
		g3d.ddAy = rotateRateOfAyInDegree*Math.PI/180.0;
		g3d.Ay= g3d.Ay + g3d.ddAy; // auto-rotate : if (ddAy==0.0), stop
		if (g3d.Ay>Math.PI) g3d.Ay = g3d.Ay - 2.0*Math.PI;
		if (g3d.Ay<-Math.PI) g3d.Ay = g3d.Ay + 2.0*Math.PI;
		g3d.setBox();           // set box apex
		g3d.setRotateXY(g3d.Ax,g3d.Ay); // set rotate param
		g3d.rotateApexXY();     // box Apex rotate--> pxApex[i],pyApex[i],pzApex[i]
		g3d.markFarEdge();      // boxEdge[iEdge][2]=1:far side edge or 0:near side edge
	};

	g3d.setBox = function() {
		for (let i=0; i<8; i++) {
			g3d.xApex[i] = g3d.boxApex[i][0]*g3d.xMax;
			g3d.yApex[i] = g3d.boxApex[i][1]*g3d.yMax;
			g3d.zApex[i] = g3d.boxApex[i][2]*g3d.zMax;
		}
	};

	g3d.setRotateXY = function(angleX,angleY) {
		g3d.cosAx = Math.cos(angleX);
		g3d.sinAx = Math.sin(angleX);
		g3d.cosAy = Math.cos(angleY);
		g3d.sinAy = Math.sin(angleY);
		g3d.cx0 = 0.5*g3d.xMax;
		g3d.cy0 = 0.5*g3d.yMax;
		g3d.cz0 = 0.5*g3d.zMax;
	};

	g3d.rotateApexXY = function() { // rotate box apex
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;

		for (let i=0; i<8; i++) {
			g3d.pxApex[i] = cosAy*(g3d.xApex[i]-cx0)+sinAy*(sinAx*(g3d.yApex[i]-cy0)+cosAx*(g3d.zApex[i]-cz0))+cx0;
			g3d.pyApex[i] = cosAx*(g3d.yApex[i]-cy0)-sinAx*(g3d.zApex[i]-cz0) + cy0;
			g3d.pzApex[i] =-sinAy*(g3d.xApex[i]-cx0)+cosAy*(sinAx*(g3d.yApex[i]-cy0)+cosAx*(g3d.zApex[i]-cz0))+cz0;
		}
	};

	g3d.markFarEdge = function() {
		//seek far apex --> iMin
		let zMin = g3d.pzApex[0];
		let iMin = 0;
		for (let i=1; i<8; i++) {
			if (zMin>g3d.pzApex[i]) {
				zMin = g3d.pzApex[i];
				iMin = i;
			}
		}
		//mark far edge
		for (let iEdge=0; iEdge<12; iEdge++) {
			g3d.boxEdge[iEdge][2] = 0;
			if (g3d.boxEdge[iEdge][0]==iMin || g3d.boxEdge[iEdge][1]==iMin) g3d.boxEdge[iEdge][2] = 1;
		}
	};

	g3d.drawRotatedDisc = function(ctx, x,y,z,r,color,sc,xp,yp) {
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;

		const x1 = cosAy*(x-cx0)+sinAy*(sinAx*(y-cy0)+cosAx*(z-cz0)) + cx0
		const y1 = cosAx*(y-cy0)-sinAx*(z-cz0) + cy0
		//z1 =-sinAy*(x-cx0)+cosAy*(sinAx*(y-cy0)+cosAx*(z-cz0)) + cz0
		g3d.drawDisc(ctx, x1*sc+xp,y1*sc+yp,r,color);
	};

	g3d.drawRotatedLine = function(ctx, x1,y1,z1,x2,y2,z2,color,sc,xp,yp) {
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;

		const x1p = cosAy*(x1-cx0)+sinAy*(sinAx*(y1-cy0)+cosAx*(z1-cz0)) + cx0
		const y1p = cosAx*(y1-cy0)-sinAx*(z1-cz0) + cy0
		const x2p = cosAy*(x2-cx0)+sinAy*(sinAx*(y2-cy0)+cosAx*(z2-cz0)) + cx0
		const y2p = cosAx*(y2-cy0)-sinAx*(z2-cz0) + cy0
		g3d.drawLine(ctx, x1p*sc+xp,y1p*sc+yp,x2p*sc+xp,y2p*sc+yp,color);
	};

	g3d.plotNearEdge = function(ctx, sc,xp,yp,color) {
		for (let iEdge=0; iEdge<12; iEdge++) {
			if (g3d.boxEdge[iEdge][2]==0) { //far edge mark = 1
				g3d.plotEdge(ctx, iEdge,sc,xp,yp,color);
			}
		}
	};

	g3d.plotFarEdge = function(ctx, sc,xp,yp,color) {
		for (let iEdge=0; iEdge<12; iEdge++) {
			if (g3d.boxEdge[iEdge][2]==1) { //far edge mark = 1
				g3d.plotEdge(ctx, iEdge,sc,xp,yp,color);
			}
		}
	};

	g3d.plotEdge = function(ctx, iEdge,sc,xp,yp,color) {
		let iApex = g3d.boxEdge[iEdge][0];
		const x1=g3d.pxApex[iApex]*sc+xp, y1=g3d.pyApex[iApex]*sc+yp;
		iApex = g3d.boxEdge[iEdge][1];
		const x2=g3d.pxApex[iApex]*sc+xp, y2=g3d.pyApex[iApex]*sc+yp;
		g3d.drawLine(ctx, x1, y1, x2, y2, color);
	};

	g3d.drawLine = function(ctx, x1, y1, x2, y2, color) {
		ctx.strokeStyle = color;
		ctx.beginPath();
		ctx.moveTo(x1, y1);
		ctx.lineTo(x2, y2);
		ctx.stroke();
	};

	g3d.drawDisc = function(ctx, x, y, r, color) {
		ctx.fillStyle = color;
		ctx.beginPath();
		ctx.arc(x, y, r, 0, 2*Math.PI, false);
		ctx.fill();
	};

	// --------------------  end of graphics 3D (field) module  --------------------

	// g3d_extension field3D  created 2023.06.04, last updated 2023.06.04
	// g3d.drawField3D( ctx, rotAngle, fieldFunc, colorMode )
	g3d.drawField3D = function(ctx, rotAngle, fieldFunc, colorMode ) {
		let sc, xp, yp; [ sc, xp, yp ] = g3d.scxpypFunc();
		const nnx=g_NNx, nny=g_NNy, nnz=g_NNz, dx=g_dx, hh=g_dx*sc;
		const threshold = g3d.drawField3D.threshold;

		g3d.set3DRotateXY(rotAngle);
		g3d.plotFarEdge(ctx, sc,xp,yp,"#444400"); // dark yellow
		for (let ii=0; ii<nnx; ii++) {
			let i=ii; if (g3d.pzApex[1]-g3d.pzApex[0]<0) i=nnx-ii-1;
			for (let jj=0; jj<nny; jj++) {
				let j=jj; if (g3d.pzApex[2]-g3d.pzApex[0]<0) j=nny-jj-1;
				for (let kk=0; kk<nnz; kk++) {
					let k=kk; if (g3d.pzApex[4]-g3d.pzApex[0]<0) k=nnz-kk-1;

					const f = fieldFunc(i,nny-j-1,k);
					const r = Math.min(Math.pow(Math.abs(f),0.333),0.6*hh);
					if ( r>threshold ) {
						let th, colr;
						if ( colorMode==0 ) { // 0:red(f>0) or blue(f<0)
							colr = (f>0) ? "#ff0000" : "#0000ff";
						} else if ( colorMode==1 ) { //
							th = 120.0-30.0*r*Math.sign(f);
							if (th>270.0) th = 270.0;
							if (th<-30.0) th = -30.0;
							colr = `hsl(${th},100%,50%)`;
						} else if ( colorMode==2 ) { // small hsl3-arg.
							th = 120.0-30.0*r*Math.sign(f);
							if (th>270.0) th = 270.0;
							if (th<-30.0) th = -30.0;
							colr = `hsl(${th},60%,30%)`;
						}
						g3d.drawRotatedDisc( ctx, i*dx, j*dx, k*dx, r*g3d.zoom, colr, sc, xp, yp );
					}
				}
			}
		}
		g3d.plotNearEdge( ctx, sc, xp, yp, "#999900"); //yellow

		ctx.font = "12px 'sans-serif'";
		ctx.fillStyle = "#888888";
		ctx.fillText(`Ax=${(g3d.Ax*180/Math.PI).toFixed(1)}, Ay=${(g3d.Ay*180/Math.PI).toFixed(1)}`, 10, 15);
		ctx.font = "16px 'sans-serif'";
	};
	g3d.drawField3D.threshold = 0.7;

	// g3d_extension grid2d  created 2023.06.01, last updated 2023.06.04
	// g3d.drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
	g3d.drawGrid2D = function ( ctx, rotAngle, zFunc, colorFactor, inc, showBox ) {
		let sc, xp, yp; [ sc, xp, yp ] = g3d.scxpypFunc();
		const nnx = g_NNx, nny = g_NNy, threshold = g3d.drawGrid2D.threshold;

		g3d.set3DRotateXY(rotAngle);
		if ( (showBox & 1)>0 || showBox==undefined ) g3d.plotFarEdge(ctx, sc,xp,yp,"#444400"); // dark yellow
		for (let jj=0; jj<nny; jj+=inc) {
			let j=jj;if (g3d.pzApex[2]-g3d.pzApex[0]<0) j=nny-jj-1;
			for (let ii=0; ii<nnx; ii++) {
				let i=ii;if (g3d.pzApex[1]-g3d.pzApex[0]<0) i=nnx-ii-1;
				if (i<0 || i+1>nnx-1) continue;
				const f = zFunc(i,nny-j-1);
				const x = i*g_dx, y = j*g_dy, z = f + g3d.cz0;
				const f1 = zFunc(i+1,nny-j-1);
				const x1 = (i+1)*g_dx, y1 = j*g_dy, z1 = f1 + g3d.cz0;
				let colr;
				if ( typeof(colorFactor)=='number' ) {
					const th = (18120 - Math.floor(colorFactor*180.0*(f+f1)/g3d.cz0))%360;
					const a = Math.abs((f+f1)/g3d.cz0);
					colr = (a>threshold) ? `hsl(${th},100%,50%)` : "#444444" ;
				} else if ( typeof(colorFactor)=='string' ) {
					colr = colorFactor;
				} else if ( typeof(colorFactor)=='function' ) {
					colr = colorFactor(i,nny-j-1);
				}
				g3d.drawRotatedLine(ctx, x,y,z,x1,y1,z1,colr,sc,xp,yp);
			}
		}
		for (let ii=0; ii<nnx; ii+=inc) {
			let i=ii;if (g3d.pzApex[1]-g3d.pzApex[0]<0) i=nnx-ii-1;
			for (let jj=0; jj<nny; jj++) {
				let j=jj;if (g3d.pzApex[2]-g3d.pzApex[0]<0) j=nny-jj-1;
				if (j<0 || j+1>nny-1) continue;
				const f = zFunc(i,nny-j-1);
				const x = i*g_dx, y = j*g_dy, z = f + g3d.cz0;
				const f1 = zFunc(i,nny-j-2);
				const x1 = i*g_dx, y1 = (j+1)*g_dy, z1 = f1 + g3d.cz0;
				let colr;
				if ( typeof(colorFactor)=='number' ) {
					const th = (18120 - Math.floor(colorFactor*180.0*(f+f1)/g3d.cz0))%360;
					const a = Math.abs((f+f1)/g3d.cz0);
					colr = (a>threshold) ? `hsl(${th},100%,50%)` : "#444444" ;
				} else if ( typeof(colorFactor)=='string' ) {
					colr = colorFactor;
				} else if ( typeof(colorFactor)=='function' ) {
					colr = colorFactor(i,nny-j-1);
				}
				g3d.drawRotatedLine(ctx, x,y,z,x1,y1,z1,colr,sc,xp,yp);
			}
		}
		if ( (showBox & 2)>0 || showBox==undefined ) g3d.plotNearEdge(ctx, sc,xp,yp,"#999900"); //yellow

		ctx.font = "12px 'sans-serif'";
		ctx.fillStyle = "#888888";
		ctx.fillText(`Ax=${(g3d.Ax*180/Math.PI).toFixed(1)}, Ay=${(g3d.Ay*180/Math.PI).toFixed(1)}`, 10, 15);
		ctx.font = "16px 'sans-serif'";
	};
	g3d.drawGrid2D.threshold = 0.005;


	// --------------------  control  --------------------

	function initDom() {
		document.getElementById("step_button").style.visibility = "hidden";
	}

	function reset() { resetFlag = true; }

	function pause() {
		let btn = document.getElementById("pause_button");

		pauseFlag = ( pauseFlag==false );
		if ( pauseFlag==false ) btn.innerHTML = "pause"; else btn.innerHTML = "go";

		if ( pauseFlag==true ) {
			document.getElementById("step_button").style.visibility = "visible";
		} else {
			document.getElementById("step_button").style.visibility = "hidden";
		}
	}

	function step() { stepFlag = true; }

	function setTheme() {  // select theme
		v_vIndex = 0 + document.getElementById("slct_theme").selectedIndex;
		resetFlag = true;
	}

	function setDispMode() {  // select dispMode
		dispMode = 0 + document.getElementById("slct_dispMode").selectedIndex;
	}

	function viewHome() {
		g3d.setRotateAngle(72,-15);
		g3d.zoom = 1.0;
	}

  // function controlled by python

  function breakLoop() {
    breakFlag = true;
  }

  function pysetTheme( theme ) {
    v_vIndex = theme
    document.getElementById("slct_theme").selectedIndex = theme;
    resetFlag = true;
  }

  function pysetDispMode( mode ) {
    dispMode = mode;
    document.getElementById("slct_dispMode").selectedIndex = mode;
  }

  function pygetData( pyMsg ) {
    document.getElementById("text_from_python").innerHTML = pyMsg;
    return [ sysTime, energy1000, aliveReplicas ];
  }


	// --------------------  public  --------------------

	return {
		main:				main,				// main()

		reset:				reset,				// reset()
		pause:				pause,				// pause()
		step:				step,				// step()

		setTheme:			setTheme,			// setTheme()
		setDispMode:		setDispMode,		// setDispMode()
		viewHome:			viewHome,			// viewHome()

    breakLoop: breakLoop, // breakLoop();
    pysetTheme: pysetTheme, // pysetTheme( theme )
    pysetDispMode: pysetDispMode, // pysetDispMode( mode )
    pygetData: pygetData, // pygetData( pyMsg ) : return [ sysTime, totalEnergy, aliveReplicas ]
	};

})(); // ====================  js034 end  ====================


const js = js034;
js.main();


// %%%%%%%%%%%%%%%%%%%%  end of javaScript  %%%%%%%%%%%%%%%%%%%%

</script>

<style type="text/css">
    body { text-align:left; color:#000000; background-color:#fff8dd; }
</style>

</head>

<body>
<p>[js034] ground state and energy - diffusion Monte-Carlo method 3D</p>
<canvas ID="canvas_box" style="background-color: #000000;" WIDTH="480" HEIGHT="480"></canvas>
<br>

<label>potential:</label>
<select id="slct_theme" onChange="js.setTheme()">
<option selected>in parabollic V(r)=0.5 r^2</option>
<option>in well V(r) = 0(r&lt;3), = 20(else)</option>
<option>H like V(r) = -1/r </option>
<option>in ring </option>
</select>
    <span style="margin-right: 30px;"></span>
<button onClick="js.reset()">reset</button>
    <span style="margin-right: 20px;"></span>
<button id="pause_button" onClick="js.pause()">pause</button>
    <span style="margin-right: 10px;"></span>
<button id="step_button" onClick="js.step()">step</button>
<br>

<label>disp mode:</label>
<select id="slct_dispMode" onChange="js.setDispMode()">
<option>2d psi(x,y,0)</option><option selected>3d replicas</option>
<option>3d psi(x,y,z)</option><option>3d density(x,y,z)</option>
<option>grid2d psi(x,y,0)</option><option>grid2d density(x,y,0)</option>
<option>grid2d Vext(x,y,0)</option>
</select>
<br>

<button onClick="js.viewHome()">return to initial view</button>
<br>

<p id="text_caption" ></p>
<hr width="480" align="left" color="#a0a0a0">
<button onClick="js.breakLoop()">animation break to END</button>
    <span style="margin-right: 50px;"></span> python msg:
<span id="text_from_python" ></span>
<br>

</body>
</html>


  ''')
  display(htm)
# end def


In [ ]:
# exec html-js code
exec_html_js()
print("--- push [animation break to END] button to end ---")

In [ ]:
# exec html-js code, and python control

import time
import numpy as np
import matplotlib.pyplot as plt


# exec html-js code
exec_html_js()
print("-- start --")
time.sleep(1)

# get data and print
for i in range(10):
  [ sysTime, totalEnergy, aliveReplicas ] = eval_js( 'js.pygetData({})'.format(i) )
  print(
    "i = {:>2},  ".format(i),
    "time = {:>7.2f} (au),  ".format(sysTime),
    "energy = {:10.6f} (au),  ".format(totalEnergy),
    "alive replicas = {:>5},  ".format(aliveReplicas) )
  time.sleep(2)
#

time.sleep(1)
# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

In [ ]:
# exec html-js code, and python control / change potential, dispMode

import time
import numpy as np
import matplotlib.pyplot as plt


themeList = [ '0: parabollic V=0.5 r^2', '1: well V(r) = 0(r<3), = 20(else)', '2: H like V(r) = -1/r', '3: ring' ]
dispModeList = [ '0: 2d psi(x,y,0)', '1: 3d replicass', '2: 3d psi(x,y,z)', '3: 3d density(x,y,z)', '4: grid2d psi(x,y,0)',
                '5: grid2d density(x,y,0)', '6: grid2d Vext(x,y,0)' ]

# exec html-js code
exec_html_js()
print("-- start --")
time.sleep(1)

# change theme and dispMode
for theme in [ 0, 1, 2, 3 ]:
  eval_js( 'js.pysetTheme({})'.format(theme) )
  print( "-- theme:", themeList[theme], "--" )
  for dispMode in [ 0, 1, 2, 3, 4, 5, 6 ]:
    eval_js( 'js.pysetDispMode({})'.format(dispMode) )
    print( "-- -- dispMode:", dispModeList[dispMode], "--" )
    # get data and display
    [ sysTime, totalEnergy, aliveReplicas ] = eval_js( 'js.pygetData(0)' )
    print(
      "\t time = {:>7.2f} (au),  ".format(sysTime),
      "energy = {:10.6f} (au),  ".format(totalEnergy),
      "alive replicas = {:>5},  ".format(aliveReplicas) )
    time.sleep(2)
time.sleep(1)

# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

In [ ]:
# exec html-js code, and python control / total energy convergence

import time
import numpy as np
import matplotlib.pyplot as plt


timeList = []
teList = []

# exec html-js code
exec_html_js()
print("-- start --")

# get data and print
for i in range(20):
  [ sysTime, totalEnergy, aliveReplicas ] = eval_js( 'js.pygetData({})'.format(i) )
  timeList.append(sysTime)
  teList.append(totalEnergy)
  if (i%2==0):
    print(
      "i = {:>2},  ".format(i),
      "time = {:>7.2f} (au),  ".format(sysTime),
      "energy = {:10.6f} (au),  ".format(totalEnergy),
      "alive replicas = {:>5},  ".format(aliveReplicas) )
  time.sleep(1)
#

time.sleep(1)
# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

#plot
plt.plot(timeList, teList,'og-')
plt.ylabel("energy (au)")
plt.xlabel("time (au)")
plt.grid()
plt.show()